In [3]:
from pyspark.resultiterable import ResultIterable
from pyspark.sql import SparkSession
from typing import Iterable, Tuple

In [4]:
def computeIndegree(urls: ResultIterable[str], rank: float) -> Iterable[Tuple[str, float]]:
    no = len(urls)
    for url in urls:
        yield (url, rank / no)

In [5]:
def partition(urls: str) -> Tuple[str, str]:
    parts = re.split(r'\s+', urls)
    return parts[0], parts[1]

In [30]:



spark = SparkSession\
    .builder\
    .appName("Pagerank using PySpark")\
    .getOrCreate()




## 1000 edges from email-Eu-core.txt

In [11]:
rows = spark.read.text("email-Eu-core.txt").rdd.map(lambda r: r[0])
i = 0

for each in rows.collect()[:1000]:
    print (str(i), ":", each)
    i += 1


0 : 0 1
1 : 2 3
2 : 2 4
3 : 5 6
4 : 5 7
5 : 8 9
6 : 10 11
7 : 12 13
8 : 12 14
9 : 15 16
10 : 17 18
11 : 12 19
12 : 20 21
13 : 20 22
14 : 23 24
15 : 23 25
16 : 23 26
17 : 23 27
18 : 23 28
19 : 23 29
20 : 23 30
21 : 23 31
22 : 23 32
23 : 23 33
24 : 23 34
25 : 23 35
26 : 23 36
27 : 23 37
28 : 23 38
29 : 23 39
30 : 23 40
31 : 41 42
32 : 43 44
33 : 45 46
34 : 47 48
35 : 49 50
36 : 41 51
37 : 52 53
38 : 54 55
39 : 54 56
40 : 54 57
41 : 54 58
42 : 54 59
43 : 60 61
44 : 54 54
45 : 62 63
46 : 64 65
47 : 62 4
48 : 66 67
49 : 68 69
50 : 42 41
51 : 70 71
52 : 72 21
53 : 71 70
54 : 73 74
55 : 75 76
56 : 75 48
57 : 77 78
58 : 77 79
59 : 80 81
60 : 80 68
61 : 19 62
62 : 82 83
63 : 22 21
64 : 82 84
65 : 21 72
66 : 41 85
67 : 41 86
68 : 41 87
69 : 82 86
70 : 88 89
71 : 90 91
72 : 92 20
73 : 41 93
74 : 41 94
75 : 41 95
76 : 89 96
77 : 89 88
78 : 97 98
79 : 97 99
80 : 97 100
81 : 97 101
82 : 97 102
83 : 103 104
84 : 51 41
85 : 82 105
86 : 90 106
87 : 62 107
88 : 108 109
89 : 108 66
90 : 108 110
91 : 108 

In [14]:
import re
import sys
import time
from typing import Iterable, Tuple
from operator import add
from pyspark.sql import SparkSession
from pyspark.resultiterable import ResultIterable

## Outgoing connections for 100 nodes

In [24]:
edges = rows.map(lambda urls: partition(urls)).distinct().groupByKey().cache()
for cn in edges.collect()[:100]:
    print("Node", cn[0], 'connets to these nodes', list(cn[1]), '\n')

Node 0 connets to these nodes ['1', '316', '146', '268', '581', '221', '18', '734', '178', '380', '0', '459', '215', '250', '148', '73', '498', '226', '101', '218', '377', '17', '74', '177', '248', '103', '560', '309', '88', '5', '297', '313', '223', '238', '368', '266', '222', '283', '6', '64', '166'] 

Node 2 connets to these nodes ['3', '4', '195', '305', '2', '412', '102', '194', '55', '252', '13', '63', '192', '586', '408', '138', '434', '6', '281', '193', '174', '285', '137', '323', '211', '58', '238', '520', '481', '665', '59', '208', '5', '506', '516', '517', '160', '286', '57', '132', '409', '411', '587', '140', '96', '64', '809', '826', '158', '738', '832', '273', '803', '845', '571', '89', '54', '300', '489', '612', '249', '865', '283', '107', '880', '698', '899', '86', '886', '162', '546', '812', '859', '308', '131', '56', '333', '971', '271', '990', '685', '405', '896', '1001'] 

Node 5 connets to these nodes ['6', '7', '215', '5', '117', '64', '327', '232', '53', '115', '

In [26]:
ranks = edges.map(lambda url_neighbors: (url_neighbors[0], 1.0))

In [27]:
import time

start = time.process_time()
itr = 0
while itr < 10:
    degree = edges.join(ranks).flatMap(lambda url_urls_rank: computeIndegree(
            url_urls_rank[1][0], url_urls_rank[1][1]
        ))
    degree = edges.fullOuterJoin(degree).mapValues(lambda x : x[1] or 0.0)

    ranks = degree.reduceByKey(add).mapValues(lambda rank: 0.15 + rank * 0.85)
    itr = itr+1
print("Time taken: ", time.process_time() - start, "sec")

Time taken:  0.1374054349999998 sec


In [28]:
ranks.coalesce(1).saveAsTextFile("pyspark_op")

In [29]:
r_list = []
for node in ranks.collect():
    r_list.append((int(node[0]), node[1]))

ranked_list = (sorted(r_list, key = lambda node: node[1], reverse = True))
# Ranking page on basis of decreasing probablities
rankk = 0
for line in ranked_list:
    print(rankk, ":", line)
    rankk = rankk + 1

with open("output_core_Ranked.txt", "w") as result:
    for line in ranked_list:
        result.write(str(line)+'\n')
spark.stop()

0 : (1, 7.171940211620537)
1 : (160, 5.67721815237069)
2 : (130, 5.310425246723762)
3 : (62, 4.474627372734478)
4 : (86, 4.30943598473273)
5 : (107, 4.208322818604381)
6 : (365, 4.035149493589312)
7 : (121, 3.9719038112276355)
8 : (5, 3.793758876302937)
9 : (129, 3.738847750763658)
10 : (183, 3.5947140720951056)
11 : (64, 3.538337568175954)
12 : (434, 3.5379543259790958)
13 : (128, 3.413549649673629)
14 : (106, 3.3340616418402362)
15 : (21, 3.163134802758971)
16 : (532, 3.1220138719278467)
17 : (166, 3.105019944275731)
18 : (301, 2.9847424451526297)
19 : (82, 2.931605044738474)
20 : (211, 2.911705062895369)
21 : (333, 2.909681190499702)
22 : (105, 2.860829774131669)
23 : (377, 2.85738586064778)
24 : (256, 2.8179154575468814)
25 : (283, 2.766569196781285)
26 : (420, 2.698559068635923)
27 : (227, 2.6946735330789875)
28 : (44, 2.6330053190282867)
29 : (87, 2.618879821097143)
30 : (820, 2.603713539982144)
31 : (96, 2.6013996992924953)
32 : (114, 2.573296425640005)
33 : (280, 2.551898496951